- ENVISIONING THE ANSWER:
- Asking Question #############
- Search for info ##########
- structure data ######
- Envision the answer
- Represent the visualizaion
- Tell The story


- Introduction ###########
- Motivation  ##########

- Question ########
- Data Set Description ########

- Data Processing ####
- Visualization/Description
- Quantitative Analysis (numbers)
- Qualitative Analysis (characters)

- Analysis

- Conclusion

INTRODUCTION/MOTIVATION:

QUESTION: (Envision: Ask Question)


- How to maximize both rating, viewership

- User Fraction of each characters dialogue, (compared to each other, and overall)

- USE WORDS, NOT SENTENCES, TO GET A MORE ACCURATE VIEW


- And the ratio of outward explamation they do (UPPER CASE, EXCLAMATION MARK)

DATA SET: (data set description, search for info, structuring????????)

In [1]:
import pandas as pd

In [6]:
dialogue = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends.csv"))

dialogue.head(5)


,text,speaker,season,episode,scene,utterance
0,There's nothing to tell! He's just some guy I ...,Monica Geller,1,1,1,1
1,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,1,1,1,2
2,"All right Joey, be nice. So does he have a hum...",Chandler Bing,1,1,1,3
3,"Wait, does he eat chalk?",Phoebe Buffay,1,1,1,4
4,"(They all stare, bemused.)",Scene Directions,1,1,1,5


In [5]:
episodes = pd.read_csv(("https://raw.githubusercontent.com/"
 "rfordatascience/tidytuesday/master/data/"
 "2020/2020-09-08/friends_info.csv"))

episodes.head(5)

,season,episode,title,directed_by,written_by,air_date,us_views_millions,imdb_rating
0,1,1,The Pilot,James Burrows,David Crane & Marta Kauffman,1994-09-22,21.5,8.3
1,1,2,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,1994-09-29,20.2,8.1
2,1,3,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,1994-10-06,19.5,8.2
3,1,4,The One with George Stephanopoulos,James Burrows,Alexa Junge,1994-10-13,19.7,8.1
4,1,5,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,1994-10-20,18.6,8.5


DATA PRE PROCESSING:

RESULTING VISUALIZATIONS:

In [29]:
#CLEANING OUT NON COMMUNICATION ROW (IE: "Setting/Noise")
verbal_dialogue = dialogue[~dialogue["speaker"].isin(["Scene Directions", "#ALL#", "NA"])]

main_character_names = ["Ross Geller", "Monica Geller",
                        "Phoebe Buffay", "Chandler Bing",
                        "Joey Tribbiani", "Rachel Green"]


def total_words(x):
    return x.str.len().sum()

def total_lines(x):
    return len(x)

def total_words_of_exclamation(x):
    return x[x.str.contains(r'[A-Z]{2,}') | x.str.contains(r'!')].str.len().sum()


In [44]:
#Obtains total WORDS + LINES per episode
total_lines_words_per_episode = verbal_dialogue.groupby(["season", "episode"])["text"].agg([total_words, total_lines])


main_character_dialogue = verbal_dialogue[verbal_dialogue["speaker"].isin(main_character_names)]
main_character_stats = main_character_dialogue.groupby(["season", "episode", "speaker"])["text"].agg([total_words, total_lines, total_words_of_exclamation]).reset_index()

other_character_dialogue = verbal_dialogue[~verbal_dialogue["speaker"].isin(main_character_names)]
other_character_stats = main_character_dialogue.groupby(["season", "episode"])["text"].agg([total_words, total_lines, total_words_of_exclamation]).reset_index()
other_character_stats.insert(2, "speaker", "Other")

other_character_stats
#main_character_stats


,season,episode,speaker,total_words,total_lines,total_words_of_exclamation
0,1,1,Other,15544,265,5120
1,1,2,Other,9738,172,2334
2,1,3,Other,11064,213,2776
3,1,4,Other,11381,217,4202
4,1,5,Other,11034,204,871
...,...,...,...,...,...,...
231,10,14,Other,12103,240,3103
232,10,15,Other,11538,199,3718
233,10,16,Other,13298,243,4930
234,10,17,Other,14551,337,5602
